In [1]:
# Load MATLAB file containing the dataset into Python
import scipy.io
matlab_file = 'url.mat'

print('Loading the MATLAB file containing the dataset...')

data = scipy.io.loadmat(matlab_file)

print('The MATLAB file has been loaded.')
print(data.keys())

Loading the MATLAB file containing the dataset...
The MATLAB file has been loaded.
dict_keys(['__header__', '__version__', '__globals__', 'FeatureTypes', 'Day120', 'Day119', 'Day118', 'Day117', 'Day116', 'Day115', 'Day114', 'Day113', 'Day112', 'Day111', 'Day110', 'Day109', 'Day108', 'Day107', 'Day106', 'Day105', 'Day104', 'Day103', 'Day102', 'Day101', 'Day100', 'Day99', 'Day98', 'Day97', 'Day96', 'Day95', 'Day94', 'Day93', 'Day92', 'Day91', 'Day90', 'Day89', 'Day88', 'Day87', 'Day86', 'Day85', 'Day84', 'Day83', 'Day82', 'Day81', 'Day80', 'Day79', 'Day78', 'Day77', 'Day76', 'Day75', 'Day74', 'Day73', 'Day72', 'Day71', 'Day70', 'Day69', 'Day68', 'Day67', 'Day66', 'Day65', 'Day64', 'Day63', 'Day62', 'Day61', 'Day60', 'Day59', 'Day58', 'Day57', 'Day56', 'Day55', 'Day54', 'Day53', 'Day52', 'Day51', 'Day50', 'Day49', 'Day48', 'Day47', 'Day46', 'Day45', 'Day44', 'Day43', 'Day42', 'Day41', 'Day40', 'Day39', 'Day38', 'Day37', 'Day36', 'Day35', 'Day34', 'Day33', 'Day32', 'Day31', 'Day30', 'Day29

In [2]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda


In [10]:

# Extract relevant information
FeatureTypes = data['FeatureTypes']

# Initialize lists to store preprocessed data
X_all = []
y_all = []

# Iterate over each day
for day in range(120):
    day_str = "Day{}".format(day)
    if day_str in data:
        X_day = data[day_str]['data'][0, 0]
        y_day = data[day_str]['labels'][0, 0]
        
        # Convert to PyTorch tensors
        X_day = torch.tensor(X_day.shape, dtype=torch.float32)
        y_day = torch.tensor(y_day.shape, dtype=torch.float32)
        
        # Normalize the data
        scaler = StandardScaler()
        X_day_normalized = scaler.fit_transform(X_day.numpy().reshape(-1, 1))
        X_day = torch.tensor(X_day_normalized, dtype=torch.float32)
        
        X_all.append(X_day)
        y_all.append(y_day)

# Concatenate data from all days
X_all_concatenated = torch.cat(X_all, dim=0)
y_all_concatenated = torch.cat(y_all, dim=0)

print('X_all_concatenated shape:', X_all_concatenated.shape)
print('y_all_concatenated shape:', y_all_concatenated.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_all_concatenated, y_all_concatenated, test_size=0.2, random_state=42, shuffle=True)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_all_concatenated shape: torch.Size([240, 1])
y_all_concatenated shape: torch.Size([240])
X_train shape: torch.Size([192, 1])
X_test shape: torch.Size([48, 1])
y_train shape: torch.Size([192])
y_test shape: torch.Size([48])


In [17]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)  # Add dropout layer with probability 0.5
        self.batch_norm1 = nn.BatchNorm1d(64)  # Add batch normalization after first linear layer
        self.batch_norm2 = nn.BatchNorm1d(64)  # Add batch normalization after second linear layer
        
    def forward(self, x):
        x = self.batch_norm1(F.relu(self.fc1(x)))  # Apply batch normalization and ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = self.batch_norm2(F.relu(self.fc2(x)))  # Apply batch normalization and ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = self.fc3(x)
        return x

net = Net().to(device)


In [18]:
# Define the loss function and optimizer
import torch.optim as optim

import torch.optim as optim

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss with Logits (for binary classification)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train the neural network
n_epochs = 200  # Number of epochs

train_losses = []  # Initialize list to store training losses
test_losses = []   # Initialize list to store testing losses
train_accuracies = []  # Initialize list to store training accuracies
test_accuracies = []   # Initialize list to store testing accuracies

for epoch in range(n_epochs):
    # Set the network to training mode
    net.train()
    
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for inputs, labels in zip(X_train, y_train):
        # Move inputs and labels to the appropriate device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        correct_train += (predicted == labels).sum().item()
        total_train += 1
    
    # Compute average training loss and accuracy for the epoch
    train_loss = running_loss / total_train
    train_accuracy = correct_train / (total_train * y_train[0].shape[0]) * 100
    
    # Set the network to evaluation mode
    net.eval()
    
    # Compute testing loss and accuracy
    test_loss = 0.0
    correct_test = 0
    total_test = 0
    
    with torch.no_grad():
        for inputs, labels in zip(X_test, y_test):
            # Move inputs and labels to the appropriate device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = net(inputs)
            loss = criterion(outputs, labels.float().view(-1, 1))
            
            # Accumulate the loss
            test_loss += loss.item()
            
            # Calculate accuracy
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_test += (predicted == labels).sum().item()
            total_test += 1
    
    # Compute average testing loss and accuracy for the epoch
    test_loss = test_loss / total_test
    test_accuracy = correct_test / (total_test * y_test[0].shape[0]) * 100
    
    # Print statistics
    print(f"Epoch {epoch + 1}/{n_epochs}, "
          f"Training Loss: {train_loss:.4f}, "
          f"Training Accuracy: {train_accuracy:.2f}%, "
          f"Testing Loss: {test_loss:.4f}, "
          f"Testing Accuracy: {test_accuracy:.2f}%")
    
    # Append training and testing losses to lists
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    # Append training and testing accuracies to lists
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

print('Finished Training')


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 128])